# Utility and Probability
## Import data and utility matrices

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
data_2022 = pd.read_csv("Game_data/2022_LoL_esports_match_data_from_OraclesElixir_20221016.csv")

/var/folders/34/kt2mvxyx0lx0f3th233kk1200000gn/T/ipykernel_17851/2514421386.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2022 = pd.read_csv("Game_data/2022_LoL_esports_match_data_from_OraclesElixir_20221016.csv")


In [11]:
def utility(path):
    util = pd.read_csv(path).T
    util.columns = util.iloc[0]
    util = util.drop(util.index[0])
    return util

In [12]:
str('Utility_matrices/'+'files')

'Utility_matrices/files'

In [13]:
# U_R = pd.read_csv("Utility/U_R.csv").T and first row is a header 
U_R = pd.read_csv("Utility/U_R.csv").T
U_R.columns = U_R.iloc[0]
U_R = U_R.drop(U_R.index[0])

U_B = pd.read_csv("Utility/U_B.csv").T
U_B.columns = U_B.iloc[0]
U_B = U_B.drop(U_B.index[0])

In [19]:
U = pd.read_csv("Utility_matrices/MAD Lions.csv").T
# only kepp last 6 rows 
U = U.iloc[-6:]
# transform row 0 to column names
U.columns = U.iloc[0]
# drop row 0
U = U.drop(U.index[0])
U

Champion,Aatrox,Ahri,Akali,Akshan,Alistar,Amumu,Aphelios,Ashe,Azir,Bard,...,Xin Zhao,Yasuo,Yone,Yuumi,Zac,Zeri,Ziggs,Zilean,Zoe,Zyra
bot,0.0,0.0,0.0,0.0,0.0,0.0,2.645751,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.828427,0.0,0.0,0.0,0.0
jng,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mid,0.0,1.732051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.414214,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
sup,0.0,0.0,0.0,0.0,1.414214,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.44949,0.0,0.0,0.0,1.0,0.0,0.0
top,2.645751,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Probability Matrices/Vectors of Picks and Bans

We want to choose a team against which our optimization model will play. Lets pick the team which has played the most game in the dataset so that the data is the most represenative.

In [20]:
# all unique league in data_2022 
leagues = data_2022['league'].unique()
leagues

array(['LCK CL', 'LPL', 'NLC', 'SL', 'Proving Grounds Circuit', 'UL',
       'PRM', 'LCK', 'LFL', 'LEC', 'LCS', 'LFL2', 'GLL', 'HM', 'ESLOL',
       'EBL', 'LPLOL', 'PGN', 'LCSA', 'LVP DDH', 'TRA', 'TCL', 'CBLOL',
       'LCO', 'LHE', 'GL', 'EL', 'CBLOLA', 'LMF', 'VL', 'LLA', 'HC',
       'LDL', 'LJL', 'PCS', 'VCS', 'UPL', 'LCL', 'NEXO', 'EM', 'LAS',
       'MSI', 'LJLA', 'CT', 'WCS'], dtype=object)

In [21]:
# all unique teams for LEC league 
LEC_teams = data_2022[data_2022['league'] == 'LEC']['teamname'].unique()
LEC_teams

array(['MAD Lions', 'Team Vitality', 'Rogue', 'SK Gaming',
       'Excel Esports', 'G2 Esports', 'Astralis', 'Misfits Gaming',
       'Fnatic', 'Team BDS'], dtype=object)

In [22]:
len(LEC_teams)

10

In [23]:
# HERE WE CAN EASILY LOOP THROUGH THE DIFFERENT TEAMS AND GET THE DATA FOR EACH TEAM
df_rngu = data_2022[data_2022['teamname'] == 'Royal Never Give Up']

In [24]:
def pre_processing(df, U):
    # how many unique champion in RNGU by alphabetical order
    champions = list(U.columns)

    # probability of chosing champion i in df_rngu
    p_pick = df.groupby('champion').count()['gameid']/len(df)

    # p_pick to dataframe
    p_pick = pd.DataFrame(p_pick).T

    # create empty dataframe with column name as champions
    pick_vec = pd.DataFrame(columns=champions)
    # add a row of 0s called probability
    pick_vec.loc['probability'] = 0

    for i in range(len(champions)):
        if champions[i] in list(p_pick.columns):
            pick_vec[champions[i]][0] = p_pick[champions[i]][0]
        else:
            pass

    # scale pick_vec so that the sum of each row is 1
    pick_vec = pick_vec.div(pick_vec.sum(axis=1), axis=0)
    # Now lets transform it into a matrix of probabilities.
    pick_mat = U.T.copy()
    for i in range(pick_mat.shape[0]):
        for j in range(pick_mat.shape[1]):
            if pick_mat.iloc[i,j] > 0:
                pick_mat.iloc[i,j] = pick_vec.iloc[:,i][0]

    # get champion names as independant strings
    df.iloc[4, 18:23].values.astype(str)

    # for all dataset, create a list with ban1, ban2, ban3, ban4, ban5
    ban_list = []
    for i in range(len(data_2022)):
        bans = data_2022.iloc[i, 18:23].values
        for j in range(len(bans)):
            ban_list.append(bans[j])
    
    # create empty dataframe with column name as champions
    ban_vec = pd.DataFrame(columns=champions)
    # add a row of 0s called probability
    ban_vec.loc['probability'] = 0

    # for every column, calculate the probability of being banned from ban_list
    for i in range(len(champions)):
        if champions[i] in ban_list:
            ban_vec[champions[i]][0] = ban_list.count(champions[i])/len(ban_list)
        else:
            pass

    # scale ban_vec so that the sum of each row is 1
    ban_vec = ban_vec.div(ban_vec.sum(axis=1), axis=0)

    # Now lets transform it into a matrix of probabilities.
    ban_mat = U.T.copy()

    for i in range(ban_mat.shape[0]):
        for j in range(ban_mat.shape[1]):
            if ban_mat.iloc[i,j] > 0:
                ban_mat.iloc[i,j] = ban_vec.iloc[:,i][0]

    return pick_mat, ban_mat, ban_vec


In [25]:
# dataset for teamname is MAD Lions
df_MadLions = data_2022[data_2022['teamname'] == 'MAD Lions']
df_Vitality = data_2022[data_2022['teamname'] == 'Team Vitality']
df_Rogue = data_2022[data_2022['teamname'] == 'Rogue']
df_SK = data_2022[data_2022['teamname'] == 'SK Gaming']
df_Excel = data_2022[data_2022['teamname'] == 'Excel Esports']
df_G2 = data_2022[data_2022['teamname'] == 'G2 Esports']
df_Astralis = data_2022[data_2022['teamname'] == 'Astralis']
df_Misfits = data_2022[data_2022['teamname'] == 'Misfits Gaming']
df_Fnatic = data_2022[data_2022['teamname'] == 'Fnatic']
df_BDS = data_2022[data_2022['teamname'] == 'Team BDS']

dfs = [df_MadLions, df_Vitality, df_Rogue, df_SK, df_Excel, df_G2, df_Astralis, df_Misfits, df_Fnatic,df_BDS]

In [26]:
import os 
for df in dfs:
    pick_mat, ban_mat, ban_vec = pre_processing(df, U)
    # export pick_mat, ban_vec and ban_mat to csv files
    path = 'Probabilities/' + df['teamname'].iloc[0]
    # if path does not exist, create it
    if not os.path.exists(path):
        os.makedirs(path)
    pick_mat.to_csv(path + '/pick_mat.csv')
    ban_mat.to_csv(path + '/ban_mat.csv')
    ban_vec.to_csv(path + '/ban_vec.csv')

/var/folders/34/kt2mvxyx0lx0f3th233kk1200000gn/T/ipykernel_17851/2036660141.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pick_vec[champions[i]][0] = p_pick[champions[i]][0]
/var/folders/34/kt2mvxyx0lx0f3th233kk1200000gn/T/ipykernel_17851/2036660141.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ban_vec[champions[i]][0] = ban_list.count(champions[i])/len(ban_list)
/var/folders/34/kt2mvxyx0lx0f3th233kk1200000gn/T/ipykernel_17851/2036660141.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-